# Train your first Deep Reinforcement Learning Agent

PYTHON 3.9

El objetivo de este notebook es crear un sistema para entrenar los algoritmos que sea claro y escalable. Así que voy a tratar el menor número de celdas posibles

In [1]:
import sys #-- lo he necesitado para importar el paquete
sys.path.append('..')
import gym_hearthstone

## Argumentos
He añadido argumentos al environment para que sea más sencillo hacer el entrenamiento:

    action_type: Define el espacio de acción que se va a crear. Ahora mismo solo hay dos implementados.
    
        * "random" -> Solo una acción possible (Tomar una acción aleatoria de las que son posibles).
        * "type"   -> 6 acciones posibles (Toma una aleatoria según el tipo de acción).
        
        
        
    reward_mode: Define el reward que devuelve el environment:
    
        * "simple"    -> Solo hay reward cuando se gana o se pierde.
        * "penalize"  -> Se penalizan las acciones que no son válidas.
        * "incentive" -> Se premian las acciones que son válidas.
        * "complex"   -> Se penalizan las acciones que no son válidas y se premian las válidas.
        
He comprobado que funciona ejecutando los steps manualmente, pero todavía me queda comprobar el dqn en el otro ordenador. Además, todavía no he implementado que no ejecute ninguna acción cuándo la acción elegida no es válida (ahora mismo penaliza y hace otra acción aleatoria en ese step).

In [3]:
import gymnasium as gym
import gym_hearthstone

env = gym.make('Hearthstone-v1', action_type = "type", reward_mode = "complex")

[fireplace.entity]: Setting up game Game(players=(Player(name='Player1', hero=None), Player(name='Player2', hero=None)))
[fireplace.entity]: Tossing the coin... Player1 wins!
[fireplace.actions]: Player(name='Player1', hero=<Hero ('Rexxar')>) triggering <TargetedAction: Summon(<Summon.CARD>=<HeroPower ('Steady Shot')>)> targeting [Player(name='Player1', hero=<Hero ('Rexxar')>)]
[fireplace.actions]: Player1 summons [<HeroPower ('Steady Shot')>]
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.actions]: Player(name='Player1', hero=<Hero ('Rexxar')>) triggering <TargetedAction: Summon(<Summon.CARD>=<Hero ('Rexxar')>)> targeting [Player(name='Player1', hero=<Hero ('Rexxar')>)]
[fireplace.actions]: Player1 summons [<Hero ('Rexxar')>]
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.entity]: Player(name='Player1', hero=<Hero ('Rexxar')>) shuffles their deck
[fireplace.card]: <Minion ('Fireworks Tech')> moves from <Zone.DECK:

HearthstoneEnv - Version 0.2.0
Step.BEGIN_MULLIGAN
Step.MAIN_ACTION


In [ ]:
import gymnasium as gym
import gym_hearthstone
from stable_baselines3 import DQN

In [ ]:
model = DQN("MultiInputPolicy", env, verbose=1)

In [ ]:
import time
import os
print(os.getpid())


steps = 100
env.reset_stats()
start = time.time()
env.reset()

model.learn(total_timesteps=steps, log_interval=4)

end = time.time()
env.display_stats()
print("Avg steps per game: " + str(steps / env.curr_episode))
print("Total running time: {:.2f} and running time per game: {:.2f}".format(end-start, (end-start) / env.curr_episode) )


env.close()

## Time
# 100 steps - 9.8s
# 1000 steps - 95.5s

## Memory
# 1000 steps - 725 Kb

## Win Rate & Reward 
# 1000 steps - 0.43 & -3
#

In [4]:
env.reset()

for _ in range(100):
    action = env.action_space.sample()  # agent policy that uses the observation and info
    print(action)
    observation, reward, terminated, info = env.step(action)

    if terminated:
        print("################################## New Game ############################################")
        observation = env.reset()

env.close()
print("Game finished !!!!!!!!!!!!!!!!!!!")

C:\Users\jesus\anaconda3\lib\site-packages\gymnasium\utils\passive_env_checker.py:190: UserWarning: WARN: Future gymnasium versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
C:\Users\jesus\anaconda3\lib\site-packages\gymnasium\utils\passive_env_checker.py:203: UserWarning: WARN: Future gymnasium versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(
[fireplace.entity]: Setting up game Game(players=(Player(name='Player1', hero=None), Player(name='Player2', hero=None)))
[fireplace.entity]: Tossing the coin... Player2 wins!
[fireplace.actions]: Player(name='Player1', hero=<Hero ('Rexxar')>) triggering <TargetedAction: Summon(<Summon.CARD>=<HeroPower ('Steady Shot')>)> targeting [Player(name='Player1', hero=<Hero ('Rexxar')>)]
[fireplace.actions]: Player1 summons [<HeroPower ('Steady 

Step.BEGIN_MULLIGAN
Step.MAIN_ACTION
5525
3073
2633
2409
7805
2844
1393
3019
2666
2
------------------------
>>> Current Step 1
------------------------

>>> possible_actions 3:[[<Move.choice: 5>, 0], [<Move.choice: 5>, 1], [<Move.choice: 5>, 2]]
>>> alreadySelectedActions 0:[]
>>> possible_actions 3:[[<Move.choice: 5>, 0], [<Move.choice: 5>, 1], [<Move.choice: 5>, 2]]
doing single action for AI[<Move.choice: 5>, 2]
3703
7851
5525
2633
2409
7805
2844
1393
3019
2666


C:\Users\jesus\anaconda3\lib\site-packages\gymnasium\utils\passive_env_checker.py:235: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(
C:\Users\jesus\anaconda3\lib\site-packages\gymnasium\utils\passive_env_checker.py:181: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
[fireplace.entity]: Setting up game Game(players=(Player(name='Player1', hero=None), Player(name='Player2', hero=None)))
[fireplace.entity]: Tossing the coin... Player2 wins!
[fireplace.actions]: Player(name='Player1', hero=<Hero ('Rexxar')>) triggering <TargetedAction: Summon(<Summon.CARD>=<HeroPower ('Steady Shot')>)> targeting [Player(name='Player1', hero=<Hero ('Rexxar')>)]
[fireplace.actions]: Player1 summons [<HeroPower ('Steady Shot')>]
[fireplace.entity]: Empty

################################## New Game ############################################
Step.BEGIN_MULLIGAN
Step.MAIN_ACTION
2620
3035
5713
7851
857
2844
4772
1143
3654
3
------------------------
>>> Current Step 1
------------------------

>>> possible_actions 3:[[<Move.choice: 5>, 0], [<Move.choice: 5>, 1], [<Move.choice: 5>, 2]]
>>> alreadySelectedActions 0:[]
>>> possible_actions 3:[[<Move.choice: 5>, 0], [<Move.choice: 5>, 1], [<Move.choice: 5>, 2]]
doing single action for AI[<Move.choice: 5>, 0]


[fireplace.entity]: Player2 begins turn 2
[fireplace.entity]: Player2 is now at 2 mana crystals
[fireplace.actions]: Player(name='Player2', hero=<Hero ('Garrosh Hellscream')>) triggering <TargetedAction: Draw()> targeting [Player(name='Player2', hero=<Hero ('Garrosh Hellscream')>)]
[fireplace.entity]: Player2 draws <Minion ('Warsong Commander')>
[fireplace.card]: <Minion ('Warsong Commander')> moves from <Zone.DECK: 2> to <Zone.HAND: 3>
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.entity]: Setting up game Game(players=(Player(name='Player1', hero=None), Player(name='Player2', hero=None)))
[fireplace.entity]: Tossing the coin... Player2 wins!
[fireplace.actions]: Player(name='Player1', hero=<Hero ('Rexxar')>) triggering <TargetedAction: Summon(<Summon.CARD>=<HeroPower ('Steady Shot')>)> targeting [Player(name='Player1', hero=<Hero ('Rexxar')>)]
[fireplace.actions]: Player1 summons [<HeroPower ('Steady Shot')>]
[fireplace.entity]: Empty stack, refres

...

4833
3746
5674
2662
5628
5524
2844
826
4864
172
################################## New Game ############################################
Step.BEGIN_MULLIGAN


[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.actions]: Player(name='Player1', hero=<Hero ('Rexxar')>) triggering <TargetedAction: Draw()> targeting [Player(name='Player1', hero=<Hero ('Rexxar')>)]
[fireplace.entity]: Player1 draws <Minion ('Toothy Chest')>
[fireplace.card]: <Minion ('Toothy Chest')> moves from <Zone.DECK: 2> to <Zone.HAND: 3>
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.card]: <Minion ('Worgen Infiltrator')> moves from <Zone.HAND: 3> to <Zone.DECK: 2>
[fireplace.entity]: Player(name='Player1', hero=<Hero ('Rexxar')>) shuffles their deck
[fireplace.actions]: Player(name='Player2', hero=<Hero ('Garrosh Hellscream')>) triggering <TargetedAction: Draw()> targeting [Player(name='Player2', hero=<Hero ('Garrosh Hellscream')>)]
[fireplace.entity]: Player2 draws <Minion ('Volatile Elemental')>
[fireplace.card]: <Minion ('Volatile Elemental')> moves from <Zone.DECK: 2> to <Zone.HAND: 3>
[fireplace.entity

Step.MAIN_ACTION
2806
1628
2638
2679
4801
2844
183
3236
163
4
------------------------
>>> Current Step 1
------------------------

>>> possible_actions 3:[[<Move.choice: 5>, 0], [<Move.choice: 5>, 1], [<Move.choice: 5>, 2]]
>>> alreadySelectedActions 0:[]
>>> possible_actions 3:[[<Move.choice: 5>, 0], [<Move.choice: 5>, 1], [<Move.choice: 5>, 2]]
doing single action for AI[<Move.choice: 5>, 1]
3736
7829
2806
1628
2679
4801
2844
183
3236
163
################################## New Game ############################################


[fireplace.actions]: Player(name='Player2', hero=<Hero ('Garrosh Hellscream')>) triggering <TargetedAction: Summon(<Summon.CARD>=<HeroPower ('Armor Up!')>)> targeting [Player(name='Player2', hero=<Hero ('Garrosh Hellscream')>)]
[fireplace.actions]: Player2 summons [<HeroPower ('Armor Up!')>]
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.actions]: Player(name='Player2', hero=<Hero ('Garrosh Hellscream')>) triggering <TargetedAction: Summon(<Summon.CARD>=<Hero ('Garrosh Hellscream')>)> targeting [Player(name='Player2', hero=<Hero ('Garrosh Hellscream')>)]
[fireplace.actions]: Player2 summons [<Hero ('Garrosh Hellscream')>]
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.entity]: Player(name='Player2', hero=<Hero ('Garrosh Hellscream')>) shuffles their deck
[fireplace.card]: <Minion ('Mana Wraith')> moves from <Zone.DECK: 2> to <Zone.HAND: 3>
[fireplace.card]: <Spell ('Commanding Shout')> moves from <Zone.DECK: 2> to 

Step.BEGIN_MULLIGAN
Step.MAIN_ACTION
4909
1993
7863
3075
149
2844
1949
5496
2731
Game finished !!!!!!!!!!!!!!!!!!!
